# Monte Carlo

In this script we will use monte carlo simulation to generate one million schedules, then rank them according to a penalty score. We will pick the five best schedules and use this for our simulation model.

In [24]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [25]:
# Define patient types. We have assumed that the schedule sets the amount of patienttypes in each shift
new = ['New'] * 5
follow_up = ['Follow-up'] * 5
returning = ['Return'] * 7

# Combine into a single list
patients = new + follow_up + returning
random.shuffle(patients)

# Number of slots
num_slots = 14


In [26]:

# The penalties are the average time spent waiting for resources in the 
# scenario over 100 simulationruns. 

# same slot penalty map
same_slot_penalty_map = {
    ('New', 'New'): 7,
    ('New', 'Follow-up'):5,
    ('New', 'Return'):6.5,
    ('Follow-up', 'Follow-up'): 19.5,
    ('Follow-up', 'New'):5,
    ('Follow-up', 'Return'):4.5,
    ('Return', 'Return'): 6,
    ('Return', 'Follow-up'):4.5,
    ('Return', 'New'):6.5,
}

# Previous slot penalty map
consecutive_slot_penalty_map = {
    ('New', 'New'): 4,
    ('New', 'Follow-up'):1.5,
    ('New', 'Return'):3,
    ('Follow-up', 'Follow-up'): 10.5,
    ('Follow-up', 'New'):3.5,
    ('Follow-up', 'Return'):2.5,
    ('Return', 'Return'): 3,
    ('Return', 'Follow-up'):1,
    ('Return', 'New'):5,
}

In [27]:
def allocate_patients(patients, num_slots):
    slots = [[] for _ in range(num_slots)]  # Initialize slots
    available_slots = list(range(num_slots)) 
    total_patients = len(patients)
    
    for i, patient in enumerate(patients):
        if i < total_patients - 3: #Allocate patient to timeslots.
            chosen_slot = np.random.choice(available_slots) # Picks a random slot
            slots[chosen_slot].append(patient)
            available_slots.remove(chosen_slot)  # Remove slot after use
        else:
            # For the last three patients, choose from any slot, allowing up to two per slot
            if i == total_patients - 3:  # Only reset available slots once
                available_slots = [idx for idx, slot in enumerate(slots) if len(slot) < 2]  # Re-add slots that are not full
            chosen_slot = np.random.choice(available_slots)
            slots[chosen_slot].append(patient)
            if len(slots[chosen_slot]) == 2:
                available_slots.remove(chosen_slot)  # Remove slot if now full

    return slots


In [28]:
# Function to calculate penalties
def calculate_penalties(slots):
    penalty_score = 0
    for i, slot in enumerate(slots):
        
        # Same slot penalty
        if len(slot) > 1:
            for j in range(len(slot)):
                for k in range(j + 1, len(slot)):
                    pair = (slot[j], slot[k])
                    if pair in same_slot_penalty_map:
                        penalty_score += same_slot_penalty_map[pair]

        # Previous slot penalty
        if i > 0:
            previous_slot = slots[i - 1]
            for patient in slot:
                for previous_patient in previous_slot:
                    consecutive_pair = (patient, previous_patient)
                    if consecutive_pair in consecutive_slot_penalty_map:
                        penalty_score += consecutive_slot_penalty_map[consecutive_pair]

    return penalty_score



In [29]:
def process_data_array_indexed(data):
    # first appointmenttime
    first_appointment = datetime.strptime("08:00:00", "%H:%M:%S")
    # Generate timestamps based on the number of unique entries
    timestamps = [first_appointment + timedelta(minutes=15 * i) for i in range(len(data) - 1)]
    
    # Convert timestamps to time-only strings
    time_strings = [ts.strftime('%H:%M:%S') for ts in timestamps]

    # add 5 hours to the appointment slots for the second shift
    time_strings_plus_5h = [(ts + timedelta(hours=5)).strftime('%H:%M:%S') for ts in timestamps]

    # Flatten the list to ensure each type has its own row but keep track of timestamps
    result = []
    for idx, patient_types in enumerate(data[1:]):
        for patient_type in patient_types:
            # Original timestamp row
            result.append((time_strings[idx], patient_type))
            # Duplicated row with timestamp + 5 hours
            result.append((time_strings_plus_5h[idx], patient_type))
    
    # Append the penalty value
    penalty_label = f'Penalty Score'
    result.append((penalty_label, data[0]))
    return result

In [30]:
# Define the number of simulations
num_simulations = 1000000

# Initialize a list to store results
results_list = []

# Run simulations
for sim in range(num_simulations):
    slots = allocate_patients(patients, num_slots) # Randomize patientplacement
    penalty = calculate_penalties(slots) #Calculate penaltyscore
    results_list.append([penalty] + list(slots))  # Combine penalty and slots into one list

# Sort the list of results based on the lowest penalties
results_list.sort(key=lambda x: x[0])
lowest_results = results_list[:5]

# Process each data set and store in individual dataframes
dataframes = []
for i, data in enumerate(lowest_results):
    processed_data = process_data_array_indexed(data)
    df = pd.DataFrame(processed_data, columns=[f'AppointmentTime_{i+1}', f'PatientType_{i+1}'])
    df = df.sort_values(by = f'AppointmentTime_{i+1}')
    df = df.reset_index(drop=True)
    dataframes.append(df)

# combine the dataframes to one
final_df_concat = pd.concat(dataframes, axis=1)
final_df_concat

,AppointmentTime_1,PatientType_1,AppointmentTime_2,PatientType_2,AppointmentTime_3,PatientType_3,AppointmentTime_4,PatientType_4,AppointmentTime_5,PatientType_5
0,08:00:00,Return,08:00:00,Return,08:00:00,Return,08:00:00,Return,08:00:00,Follow-up
1,08:00:00,Follow-up,08:00:00,Follow-up,08:00:00,Follow-up,08:00:00,Follow-up,08:00:00,Return
2,08:15:00,Return,08:15:00,Return,08:15:00,Return,08:15:00,New,08:15:00,Return
3,08:30:00,Return,08:30:00,New,08:30:00,New,08:30:00,Follow-up,08:30:00,Return
4,08:45:00,New,08:45:00,New,08:45:00,Return,08:45:00,Return,08:45:00,Follow-up
5,09:00:00,Follow-up,09:00:00,New,09:00:00,New,09:00:00,Follow-up,09:00:00,Return
6,09:15:00,New,09:15:00,Follow-up,09:15:00,Follow-up,09:15:00,New,09:15:00,Follow-up
7,09:30:00,Follow-up,09:30:00,Return,09:30:00,Return,09:30:00,New,09:15:00,Return
8,09:45:00,Return,09:45:00,Follow-up,09:30:00,Return,09:45:00,New,09:30:00,New
9,09:45:00,Return,10:00:00,Return,09:45:00,Follow-up,10:00:00,Follow-up,09:45:00,New


In [32]:
#remove penalty row
final_df = final_df_concat.drop(34)
# Save DataFrame to a text file, using a tab as a delimiter so it can be used in JaamSim
file_path = 'OptimizedSchedules.txt'
with open(file_path, 'w') as f:
    # Write the custom header and # needed for jaamsim to read the file properly
    f.write('#Schedules\n\n')
    f.write('# ')

    # Append DataFrame to the file
    final_df.to_csv(f, sep='\t', index=False)
